In [1]:
import random
import string
import time
import base64
from hashlib import sha256
from hmac import HMAC

import http.client
import requests
import pandas as pd 

APPID = "tubmafwrzhpgfiuf"
SECRET = "eotpcqbvhycdshwscqnytiwzbgonposs"


def generate_nonce(length=32):
    """Generate a random nonce."""
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))


def get_timestamp():
    """Get the current timestamp."""
    return int(time.time())


def build_sign_str(appid, nonce, timestamp):
    """Build the string to be signed."""
    return f'appid={appid}&nonce={nonce}&timestamp={timestamp}'


def calculate_signature(secret, message):
    """Calculate the HMAC SHA-256 signature."""
    return base64.urlsafe_b64encode(HMAC(secret.encode('utf-8'), message.encode('utf-8'), sha256).digest()).decode('utf-8')


def fetch_indicator_details(indicator_id):
    """Fetch the details for a specific indicator ID."""
    nonce = generate_nonce()
    timestamp = get_timestamp()
    sign_str = build_sign_str(APPID, nonce, timestamp)
    signature = calculate_signature(SECRET, sign_str)

    headers = {
        'nonce': nonce,
        'timestamp': str(timestamp),
        'appid': APPID,
        'signature': signature,
        'Accept': "*/*",
        'Accept-Encoding': "gzip, deflate, br",
        'User-Agent': "PostmanRuntime-ApipostRuntime/1.1.0",
        'Connection': "keep-alive",
    }

    url = f"https://etahub.hzinsights.com/v1/edb/data?EdbCode={indicator_id}"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json().get('Data')
    else:
        print(
            f"Failed to fetch data for ID {indicator_id}, status code: {response.status_code}")
        return None

def fetch_indicator_name(indicator_id):
    """Fetch the name for a specific indicator ID."""
    nonce = generate_nonce()
    timestamp = get_timestamp()
    sign_str = build_sign_str(APPID, nonce, timestamp)
    signature = calculate_signature(SECRET, sign_str)

    headers = {
        'nonce': nonce,
        'timestamp': str(timestamp),
        'appid': APPID,
        'signature': signature,
        'Accept': "*/*",
        'Accept-Encoding': "gzip, deflate, br",
        'User-Agent': "PostmanRuntime-ApipostRuntime/1.1.0",
        'Connection': "keep-alive",
    }

    url = f"https://etahub.hzinsights.com/v1/edb/detail?EdbCode={indicator_id}"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json().get('Data').get('EdbName')
    else:
        print(
            f"Failed to fetch data for ID {indicator_id}, status code: {response.status_code}")
        return None

def main():
    # List of indicator IDs you want to fetch
    indicator_ids = ['GO10SGCKMc1','WTICALMc1','C2503171046266802',
                     'C2504161308433420','C2210242761',
                     'C2303166303','C2411271312440786','C2504140937125405','C2504211533207730','C2504211534519556','C2504211542204938',	
                     'C2401022662','C2304100068','W8864033']  # Add more IDs as needed

    # Dictionary to store DataFrames for each indicator
    data_frames = {}

    for indicator_id in indicator_ids:
        data = fetch_indicator_details(indicator_id)
        if data:
            # Create a DataFrame with DataTime as index
            df = pd.DataFrame(data)
            df['DataTime'] = pd.to_datetime(df['DataTime'])
            df.set_index('DataTime', inplace=True)
            df.sort_index(inplace=True)
            # Only keep the 'Value' column and rename it to the indicator ID
            df = df[['Value']].rename(columns={'Value': fetch_indicator_name(indicator_id)})
            data_frames[indicator_id] = df

    # Concatenate all DataFrames along the columns
    if data_frames:
        result_df = pd.concat(data_frames.values(), axis=1)
        print(result_df.info())
        result_df.to_excel("data_input/新加坡10ppm.xlsx")
        print("Data saved successfully as '新加坡10ppm.xlsx'")



if __name__ == "__main__":
    main()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8575 entries, 1992-09-24 to 2025-11-30
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   新加坡10ppm柴油裂解       3026 non-null   float64
 1   WTI连1合约价格（路透）      2942 non-null   float64
 2   新加坡高硫燃料油380裂解价差    1926 non-null   float64
 3   LME铜价/WTI连1合约价格    2876 non-null   float64
 4   欧洲天然气制氢成本（美元/吨氢气）  2830 non-null   float64
 5   中美经济预期差指数          5635 non-null   float64
 6   美国经济惊喜指数超季节性/3年    5349 non-null   float64
 7   Brent-Dubai/7DMA   2091 non-null   float64
 8   10年期美国国债收益率同差      8285 non-null   float64
 9   美债10-2年国债收益率利差同差   7783 non-null   float64
 10  南华工业品指数超季节性/3年     4724 non-null   float64
 11  WTI M1-M4          3513 non-null   float64
 12  美国馏分燃料油库存超季节性/3年   1638 non-null   float64
 13  中国柴油出口计划量          40 non-null     float64
dtypes: float64(14)
memory usage: 1004.9 KB
None


Data saved successfully as '新加坡10ppm.xlsx'
